In [1]:
import numpy as np
import psycopg2
import psycopg2.extras
from psycopg2 import connect
from datetime import datetime, timedelta

temp_array=[] # Auswahl der Grenzbereiche der Druck Temperatur abfrage

def Datenbankauslese(Zeitraum):
    # Extrahiere die Druck-Temperatur-Werte aus einem Zeitraum, damit müssen wir weniger Abfragen machen
    # Die Zuweisung zu den Messungen folgt später

    db_cursor = db_con.cursor(cursor_factory=psycopg2.extras.DictCursor)
    db_cursor.execute("SELECT * FROM detector_01_tp WHERE timestamp > '{:%Y-%m-%d %H:%M}' AND timestamp < '{:%Y-%m-%d %H:%M}'".format(Zeitraum[0], Zeitraum[1]))

    data = db_cursor.fetchall()
    timestamp, pressure, temperature = zip(*data)

    t_over_p = np.asarray([(t*0.01+273.15)/(p*0.01) for t,p in zip(temperature, pressure)])
    return t_over_p,timestamp

# Idee: WIr lesen die Dateinamen der files aus und wandeln sie in Timestamps um. Diese werden dann zeitlich geordnet und in Dateinamen zurückumgewandelt
# Datenformat: yyyymmdd_hhmmss (2018-01-01 12:00:00 <-> 20180101_120000)




In [10]:
import os
import re
from datetime import datetime,timedelta



def mittelwerte_berechnen(timearray, pTarray, spektren_times):
    gemittelte_werte = []
    
    for spektren_time in spektren_times:
        # Finde alle Einträge im timearray, die bis zu 90 Sekunden vor spektren_time liegen
        relevante_werte = [pTarray[i] for i, t in enumerate(timearray) if t <= spektren_time and t >= spektren_time - timedelta(seconds=90)]
        
        if relevante_werte:
            # Berechne den Mittelwert der relevanten Werte
            mittelwert = np.mean(relevante_werte)
        else:
            # Wenn keine relevanten Werte gefunden wurden, setze den Mittelwert auf None oder 0
            mittelwert = None
        
        gemittelte_werte.append(mittelwert)
    
    return gemittelte_werte

def extract_timestamp(filename, folder_path):
    # Muster für die erste Klasse von Dateinamen (ddmmyyyy_hhmmss.txt)
    match1 = re.match(r'(\d{2})(\d{2})(\d{4})_(\d{2})(\d{2})(\d{2}).txt', filename)
    if match1:
        month, day, year, hour, minute, second = match1.groups()
        return datetime(
            int(year), int(month), int(day),
            int(hour), int(minute), int(second)
        )
    
    # Muster für die zweite Klasse von Dateinamen (Feld_Einstellung.txt)
    match2 = re.match(r'Feld_.*\.txt', filename)
    if match2:
        # Verwende das Erstellungsdatum der Datei
        file_path = os.path.join(folder_path, filename)
        creation_time = os.path.getctime(file_path)
        return datetime.fromtimestamp(creation_time)
    
    # Wenn kein Muster passt, None zurückgeben
    return None

def sort_files_by_creation_time(folder_path):
    # Liste aller .txt-Dateien im angegebenen Ordner
    all_files = [file for file in os.listdir(folder_path) if file.endswith('.txt')]
    
    # Extrahiere die Zeitstempel und sortiere die Dateien
    files_with_timestamps = [(file, extract_timestamp(file, folder_path)) for file in all_files]
    files_with_timestamps = [item for item in files_with_timestamps if item[1] is not None]
    files_with_timestamps.sort(key=lambda x: x[1])
    
    # Sortierte Dateinamen zurückgeben
    sorted_files = [item[0] for item in files_with_timestamps]
    timestamps=[item[1] for item in files_with_timestamps]
    return sorted_files,timestamps

# Beispielaufruf der Funktion
folder_path = 'Testfiles/Transferfeld 1'
sorted_files,timestamps = sort_files_by_creation_time(folder_path)
print("Sortierte Dateien:", sorted_files)
print("Timestamps", timestamps)


def Spektrenverarbeitung(folder_path ,timearray,pTarray):
    sorted_spectra_files,Entstehungszeit = sort_files_by_creation_time(folder_path)

Sortierte Dateien: ['09142024_152710.txt', '09142024_153216.txt', '09142024_153748.txt', '09142024_153840.txt', '09142024_154031.txt', '09142024_154221.txt', '09142024_154413.txt', '09142024_154604.txt', '09142024_154755.txt', '09142024_154946.txt', '09142024_155137.txt', '09142024_155329.txt', '09142024_155521.txt', '09142024_155713.txt', '09142024_155904.txt', '09142024_160056.txt', '09142024_160248.txt', '09142024_160440.txt', '09142024_160632.txt', '09142024_160823.txt', '09142024_161016.txt', '09142024_161207.txt', '09142024_161401.txt', '09142024_161553.txt', '09142024_161745.txt', '09142024_161938.txt', '09142024_162131.txt', '09142024_162322.txt', '09142024_162513.txt', '09142024_162704.txt', '09142024_162856.txt', '09142024_163047.txt', '09142024_163238.txt', '09142024_163429.txt', '09142024_163621.txt', '09142024_163812.txt', '09142024_164004.txt', '09142024_164157.txt', '09142024_164348.txt', '09142024_164540.txt', '09142024_164732.txt', '09142024_164924.txt', '09142024_1651